# Find threshold for deepseg_sc

This notebook runs an algorithm that finds the best combination of threshold for each contrast, by minimizing the outpout CSA associated with each threshold.

In [ ]:
import os
import numpy as np
import pandas as pd
import itertools

In [229]:
folder_csv = '/Volumes/projects/sct_deepseg_threshold/20191009'

thr = [0.0, 0.1, 0.2, 0.3, 0.5, 0.7, 0.9]  # thresholds to test
contrasts = ['T1', 'T2', 'T2s', 'MTon', 'DWI']  # various contrasts
sub = range(50)  # number of subjects

# Build csa array
csa = np.zeros([len(sub), len(contrasts), len(thr)])

# Populate csa array by reading the csv files across contrasts and thresholds
for (i_thr, i_contrast) in [(x, y) for x in range(len(thr)) for y in range(len(contrasts))]:
    csv_file = os.path.join(folder_csv, 'csa-{}_{}.csv'.format(contrasts[i_contrast], thr[i_thr]))
    df = pd.read_csv(csv_file)
    # Loop across subjects (across rows)
    for i_sub in sub:
        try:
            csa[i_sub][i_contrast][i_thr] = df['MEAN(area)'][i_sub]
        except ValueError:
            csa[i_sub][i_contrast][i_thr] = np.nan
            print("None: i_sub={}, i_thr={}, i_contrast={}".format(i_sub, i_thr, i_contrast))

None: i_sub=25, i_thr=0, i_contrast=2
None: i_sub=25, i_thr=0, i_contrast=3
None: i_sub=25, i_thr=0, i_contrast=4
None: i_sub=25, i_thr=1, i_contrast=2
None: i_sub=25, i_thr=1, i_contrast=3
None: i_sub=25, i_thr=1, i_contrast=4
None: i_sub=25, i_thr=2, i_contrast=2
None: i_sub=25, i_thr=2, i_contrast=3
None: i_sub=25, i_thr=2, i_contrast=4
None: i_sub=25, i_thr=3, i_contrast=2
None: i_sub=25, i_thr=3, i_contrast=3
None: i_sub=25, i_thr=3, i_contrast=4
None: i_sub=25, i_thr=4, i_contrast=2
None: i_sub=25, i_thr=4, i_contrast=3
None: i_sub=25, i_thr=4, i_contrast=4
None: i_sub=25, i_thr=5, i_contrast=2
None: i_sub=25, i_thr=5, i_contrast=3
None: i_sub=25, i_thr=5, i_contrast=4
None: i_sub=25, i_thr=6, i_contrast=2
None: i_sub=25, i_thr=6, i_contrast=3
None: i_sub=25, i_thr=6, i_contrast=4


In [230]:
thr_sub = []

# Loop across subjects
for i_sub in sub:
    csa_all = []
    ind_all = []
    # Loop across all possible permutations of thresholds and contrasts
    for i_thr in list(itertools.permutations(range(len(thr)), len(contrasts))):
        # Compute the STD of CSA across contrasts, for this particular combinations of thresholds
        csa_all.append(np.std([csa[i_sub][x][i_thr[x]] for x in range(len(contrasts))]))
        ind_all.append(i_thr)
#         print(csa_all)
    # Find the minimum across all CSA_STD, for this particular subject
    thr_sub.append([thr[x] for x in ind_all[np.argmin(csa_all)]])

# Average the threshold per contrast
# print(thr_sub)
np.mean(thr_sub, axis=0)

array([0.364, 0.386, 0.858, 0.228, 0.028])